# Lab: Practice the FPGA Development Flow

##### Sections

- [Development Flow for Using oneAPI with Intel® FPGAs](#Development-Flow-for-Using-oneAPI-with-Intel®-FPGAs)
- [Anatomy of a Compilation Command](#Anatomy-of-a-Compilation-Command)
- [Stage 1: Emulation](#Stage-1:-Emulation)
- [Stage 2: Optimization Report Generation](#Stage-2:-Optimization-Report-Generation)
- [Stage 3: Full Compile](#Stage-3:-Full-Compile)
- [To Learn More](#To-Learn-More)

## Learning Objectives

* Understand the development flow for Intel® FPGAs with the Intel® oneAPI Toolkits
* Practice using the flow
* Know where to go to continue your learning


***
# Development Flow for Using oneAPI with Intel® FPGAs

<img src="Assets/fpga_flow.png">

The development flow for Intel FPGAs with oneAPI involves several stages. The purpose of these stages is so that you can
* Ensure functionality of your code (you get the correct answers from your computation)
* Ensure the custom hardware built to implement your code has optimal performance

Without having to endure the lengthy compile to a full FPGA executable each time.

In this lab, we will practice the 3 stages of the flow - emulating your code to make sure your code is function, compiling to an early design representation to generate an optimization report, and compiling to an executable that includes an FPGA bitstream. The last stage will be done like a cooking show - I've got the bitstream fully compiled already to demonstrate.

***
# Anatomy of a Compilation Command

<img src="Assets/command_anatomy.png">

***
# Stage 1: Emulation

__Seconds of Compilation__

<img src="Assets/emulation.png">

The first stage of development for FPGAs with oneAPI is __emulation__. The purpose of emulation is to make sure that your code is __functional__, or in other words, that you __get the correct answers from your computations__.

For this stage, your kernel will be compiled into an x86 executable that will be run on the host. Since it is a software compile, the compile time for this stage will be very quick, usually seconds.

This quick compile time allows you to iterate through this stage many times, until your code is functionally correct.

Some more things that are quick and easy during emulation:
* Identify quickly syntax and pointer implementation errors
* Ability to debug with GDB, even within the kernel scope
* Functional debug of SYCL code with FPGA extensions

Emulation is enabled by choosing a special FPGA emulator device as your device selector within your DPC++ code. The snippet below shows how we recommend writing your DPC++ code to easily allow compiling either for a real FPGA or an emulation device.

<img src="Assets/emulation_code.png">

The compilation command used is shown below.

<img src="Assets/emulation_command.png">

__Now, let's give it a try!__

The code below implements a simple cumulative sum on an array of values.

The code is heavily commented, so please look back at it later if you'd like to understand what it is doing. (Also - keep in mind this is a simple example. It wouldn't be worth it to ever use a lookaside acceleator to sum 1024 integers!)

__First, write the code to a file__

Things to notice in the code:
* Lines 39-43: Pre-compiler directive determining whether this will be compiled for emulation or a true bitstream
* Lines 81-85: The vector add kernel that is the portion of code that would be compiled for the FPGA in a full bitstream compile

__Before we write the file, do a clean start of the Python kernel to prevent any issues with the notebook.__

In [ ]:
import os
os._exit(00)

In [ ]:
%%writefile lab/fpga_compile.cpp
//==============================================================
// Copyright Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <CL/sycl.hpp>
#include <sycl/ext/intel/fpga_extensions.hpp>
#include <iostream>
#include <vector>

// dpc_common.hpp can be found in the dev-utilities include folder.
// e.g., $ONEAPI_ROOT/dev-utilities//include/dpc_common.hpp
#include "dpc_common.hpp"

using namespace sycl;

// Vector size for this example
constexpr size_t kSize = 1024;

// Forward declare the kernel name in the global scope to reduce name mangling. 
// This is an FPGA best practice that makes it easier to identify the kernel in 
// the optimization reports.
class VectorAdd;


int main() {

  // Set up three vectors and fill two with random values.
  std::vector<int> vec_a(kSize), vec_b(kSize), vec_r(kSize);
  for (int i = 0; i < kSize; i++) {
    vec_a[i] = rand();
    vec_b[i] = rand();
  }

  // Select either:
  //  - the FPGA emulator device (CPU emulation of the FPGA)
  //  - the FPGA device (a real FPGA)
#if defined(FPGA_EMULATOR)
  ext::intel::fpga_emulator_selector device_selector;
#else
  ext::intel::fpga_selector device_selector;
#endif

  try {

    // Create a queue bound to the chosen device.
    // If the device is unavailable, a SYCL runtime exception is thrown.
    queue q(device_selector, dpc_common::exception_handler);

    // Print out the device information.
    std::cout << "Running on device: "
              << q.get_device().get_info<info::device::name>() << "\n";

    {
      // Create buffers to share data between host and device.
      // The runtime will copy the necessary data to the FPGA device memory
      // when the kernel is launched.
      buffer buf_a(vec_a);
      buffer buf_b(vec_b);
      buffer buf_r(vec_r);


      // Submit a command group to the device queue.
      q.submit([&](handler& h) {

        // The SYCL runtime uses the accessors to infer data dependencies.
        // A "read" accessor must wait for data to be copied to the device
        // before the kernel can start. A "write no_init" accessor does not.
        accessor a(buf_a, h, read_only);
        accessor b(buf_b, h, read_only);
        accessor r(buf_r, h, write_only, no_init);

        // The kernel uses single_task rather than parallel_for.
        // The task's for loop is executed in pipeline parallel on the FPGA,
        // exploiting the same parallelism as an equivalent parallel_for.
        //
        // The "kernel_args_restrict" tells the compiler that a, b, and r
        // do not alias. For a full explanation, see:
        //    DPC++FPGA/Tutorials/Features/kernel_args_restrict
        h.single_task<VectorAdd>([=]() [[intel::kernel_args_restrict]] {
          for (int i = 0; i < kSize; ++i) {
            r[i] = a[i] + b[i];
          }
        });
      });

      // The buffer destructor is invoked when the buffers pass out of scope.
      // buf_r's destructor updates the content of vec_r on the host.
    }

    // The queue destructor is invoked when q passes out of scope.
    // q's destructor invokes q's exception handler on any device exceptions.
  }
  catch (sycl::exception const& e) {
    // Catches exceptions in the host code
    std::cerr << "Caught a SYCL host exception:\n" << e.what() << "\n";

    // Most likely the runtime couldn't find FPGA hardware!
    if (e.get_cl_code() == CL_DEVICE_NOT_FOUND) {
      std::cerr << "If you are targeting an FPGA, please ensure that your "
                   "system has a correctly configured FPGA board.\n";
      std::cerr << "Run sys_check in the oneAPI root directory to verify.\n";
      std::cerr << "If you are targeting the FPGA emulator, compile with "
                   "-DFPGA_EMULATOR.\n";
    }
    std::terminate();
  }

  // Check the results.
  int correct = 0;
  for (int i = 0; i < kSize; i++) {
    if ( vec_r[i] == vec_a[i] + vec_b[i] ) {
      correct++;
    }
  }

  // Summarize and return.
  if (correct == kSize) {
    std::cout << "PASSED: results are correct\n";
  } else {
    std::cout << "FAILED: results are incorrect\n";
  }

  return !(correct == kSize);
}

__Now, we will compile the code using the fpga_emulator_selector.__

In [ ]:
! dpcpp -fintelfpga -DFPGA_EMULATOR lab/fpga_compile.cpp -o bin/fpga_compile.emu

In [ ]:
! bin/fpga_compile.emu

__You should have seen output that looked like the below:__

Running on device: Intel(R) FPGA Emulation Device<br>
PASSED: results are correct

__It's always useful to see what happens when things don't go perfectly.__

When you have time, go back to the code you just executed and introduce a syntax error (or a few). Then, click ▶ for the section of the notebook with the code, and ▶ for the section of the notebook to compile and execute the code with the FPGA emulator.

__You can see how fast and easy emulating your code is!__

That was fast, like the software compiles most software developers are used to! This fast compile and execution are why you stay at this stage until your code is functional. (ie - You're getting the correct answers from your code!)

***
## Stage 2: Optimization Report Generation
__Minutes of Compilation__

<img src="Assets/optimization_report_stage.png">

In this next section of the lab, you will compile the kernel using command line options to create an optimization report.

The optimization report is an HTML report that will be quickly generated, and give you information to guide your optimization efforts.

More specifically, the report will provide information to
* Identify any memory, performance, data-flow bottlenecks in their design
* Receive suggestions for optimization techniques to resolve said bottlenecks
* Get area and timing estimates of their designs for the desired FPGA

For this part of the lab, we'll need to work with a more complicated piece of code to demonstrate the usefulness of the optimization report. A very high-level explanation of what the code is doing is explained below. If you'd like to learn more, you can download read this <a href="Assets/hough_explanation.pdf">document</a>. Or, you can just simply think of this as a convenient piece of code at the correct difficulty level to demonstrate optimization.

The Hough Transform is a computer algorithm that transforms pixels into votes for lines. It is used as a step in edge detection.

**Image**

<img src="Assets/pic.bmp" width="300">

**Line Votes**

<img src="Assets/line_votes.png" width="300">

The command to compile to an early image with an optimization report output is shown below.

<img src="Assets/optimization_command.png">

__Now, let's try it.__

__First, write the program to a file.__

In [ ]:
%%writefile lab/hough_transform.cpp
//==============================================================
// Copyright © 2021 Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================

#include <vector>
#include <CL/sycl.hpp>
#include <sycl/ext/intel/fpga_extensions.hpp>
#include <chrono>
#include <fstream>

// This file defines the sin and cos values for each degree up to 180
#include "sin_cos_values.h"

#define WIDTH 180
#define HEIGHT 120
#define IMAGE_SIZE WIDTH*HEIGHT
#define THETAS 180
#define RHOS 217 //Size of the image diagonally: (sqrt(180^2+120^2))
#define NS (1000000000.0) // number of nanoseconds in a second

using namespace std;
using namespace sycl;

// This function reads in a bitmap and outputs an array of pixels
void read_image(char *image_array);

class Hough_Transform_kernel;

int main() {

  //Declare arrays
  char pixels[IMAGE_SIZE];
  short accumulators[THETAS*RHOS*2];

  //Initialize the accumulators
  fill(accumulators, accumulators + THETAS*RHOS*2, 0);

  //Read bitmap
  //Read in the bitmap file and get a vector of pixels
  read_image(pixels);

  //Block off this code
  //Putting all SYCL work within here ensures it concludes before this block
  //  goes out of scope. Destruction of the buffers is blocking until the
  //  host retrieves data from the buffer.
  {
    //Profiling setup
    //Set things up for profiling at the host
    chrono::high_resolution_clock::time_point t1_host, t2_host;
    event queue_event;
    cl_ulong t1_kernel, t2_kernel;
    double time_kernel;
    auto property_list = sycl::property_list{sycl::property::queue::enable_profiling()};

    //Buffer setup
    //Define the sizes of the buffers
    //The sycl buffer creation expects a type of sycl:: range for the size
    range<1> num_pixels{IMAGE_SIZE};
    range<1> num_accumulators{THETAS*RHOS*2};
    range<1> num_table_values{180};

    //Create the buffers which will pass data between the host and FPGA
    sycl::buffer<char, 1> pixels_buf(pixels, num_pixels);
    sycl::buffer<short, 1> accumulators_buf(accumulators,num_accumulators);
    sycl::buffer<float, 1> sin_table_buf(sinvals,num_table_values);
    sycl::buffer<float, 1> cos_table_buf(cosvals,num_table_values);
  
    //Device selection
    //We will explicitly compile for the FPGA_EMULATOR, CPU_HOST, or FPGA
    #if defined(FPGA_EMULATOR)
      ext::intel::fpga_emulator_selector device_selector;
    #else
      ext::intel::fpga_selector device_selector;
    #endif

    //Create queue
    sycl::queue device_queue(device_selector,NULL,property_list);
  
    //Query platform and device
    sycl::platform platform = device_queue.get_context().get_platform();
    sycl::device device = device_queue.get_device();
    std::cout << "Platform name: " <<  platform.get_info<sycl::info::platform::name>().c_str() << std::endl;
    std::cout << "Device name: " <<  device.get_info<sycl::info::device::name>().c_str() << std::endl;

    //Device queue submit
    queue_event = device_queue.submit([&](sycl::handler &cgh) {
      //Uncomment if you need to output to the screen within your kernel
      //sycl::stream os(1024,128,cgh);
      //Example of how to output to the screen
      //os<<"Hello world "<<8+5<<sycl::endl;
    
      //Create accessors
      auto _pixels = pixels_buf.get_access<sycl::access::mode::read>(cgh);
      auto _sin_table = sin_table_buf.get_access<sycl::access::mode::read>(cgh);
      auto _cos_table = cos_table_buf.get_access<sycl::access::mode::read>(cgh);
      auto _accumulators = accumulators_buf.get_access<sycl::access::mode::read_write>(cgh);

      //Call the kernel
      cgh.single_task<class Hough_Transform_kernel>([=]() [[intel::kernel_args_restrict]] {
        for (uint y=0; y<HEIGHT; y++) {
          for (uint x=0; x<WIDTH; x++){
            unsigned short int increment = 0;
            if (_pixels[(WIDTH*y)+x] != 0) {
              increment = 1;
            } else {
              increment = 0;
            }
            for (int theta=0; theta<THETAS; theta++){
              int rho = x*_cos_table[theta] + y*_sin_table[theta];
              _accumulators[(THETAS*(rho+RHOS))+theta] += increment;
            }
          }
        }
   
      });
  
    });

    //Wait for the kernel to get finished before reporting the profiling
    device_queue.wait();

    // Report kernel execution time and throughput
    t1_kernel = queue_event.get_profiling_info<sycl::info::event_profiling::command_start>();
    t2_kernel = queue_event.get_profiling_info<sycl::info::event_profiling::command_end>();
    time_kernel = (t2_kernel - t1_kernel) / NS;
    std::cout << "Kernel execution time: " << time_kernel << " seconds" << std::endl;
  }

  //Test the results against the golden results
  ifstream myFile;
  myFile.open("golden_check_file.txt",ifstream::in);
  ofstream checkFile;
  checkFile.open("compare_results.txt",ofstream::out);
  vector<int> myList;

  int number;
  while (myFile >> number) {
    myList.push_back(number);
  }

  bool failed = false;
  for (int i=0; i<THETAS*RHOS*2; i++) {
    if ((myList[i]>accumulators[i]+1) || (myList[i]<accumulators[i]-1)) {
      failed = true;
      checkFile << "Failed at " << i << ". Expected: " << myList[i] << ", Actual: "
        << accumulators[i] << std::endl;
    }
  }

  myFile.close();
  checkFile.close();

  if (failed) {printf("FAILED\n");}
  else {printf("VERIFICATION PASSED!!\n");}

  return 1;


}

/* This function reads in a bitmap file and puts it into a vector for processing */

//Struct of 3 bytes for R,G,B components
typedef struct __attribute__((__packed__)) {
  unsigned char  b;
  unsigned char  g;
  unsigned char  r;
} PIXEL;

void read_image(char *image_array) {
  //Declare a vector to hold the pixels read from the image
  //The image is 720x480 so the CPU runtimes are not too long for emulation
  PIXEL im[WIDTH*HEIGHT];

  //Open the image file for reading
  ifstream img;
  img.open("pic.bmp",ios::in);

  //The next part reads the image file into memory
  
  //Bitmap files have a 54-byte header. Skip these bits
  img.seekg(54,ios::beg);
    
  //Loop through the img stream and store pixels in an array
  for (uint i = 0; i < WIDTH*HEIGHT; i++) {
    img.read(reinterpret_cast<char*>(&im[i]),sizeof(PIXEL));

    //The image is black and white (passed through a Sobel filter already)
    //Store 1 in the array for a white pixel, 0 for a black pixel
    if (im[i].r==0 && im[i].g==0 && im[i].b==0) {
      image_array[i] = 0;
    } else {
      image_array[i] = 1;
    }

  }

}

__Now, we'll run the command to compile to an early image and take a look at the resulting optimization report.__

__This compilation takes a couple of minutes.__

In [ ]:
! dpcpp -fintelfpga lab/hough_transform.cpp -fsycl-link=early -Xshardware -o bin/hough_transform.a
! echo "The compile is finished."

__When you see "The compile is finished." above, an optimization report file will have been generated for the code.__

__Run the following command to zip the optimization report so you can look at it locally.__
__Important note: The optimization reports will show up as empty in the JupyterLab environment. They must be looked at locally.__

In [ ]:
! cd bin/hough_transform.prj; zip -r ../../report.zip reports > /dev/null
! echo "The zipped report file will appear in the file browser pane."

__Now, let's examine that report file.__

Within the Jupyter Lab environment, you will see a file browser pane on the left side. Right-click the file report.zip and select download.

On your local machine, put the report.zip file in a working directory, and unzip it. Then, double-click on the file report.html.

It should look like the picture shown below.

<img src="Assets/report_image.png">

In the report, navigate to the **Loops Analysis** section of the report by pulling down the **Throughput Analysis** menu at the top. Select Hough_Transform_kernel.B1 in the menu at the left.

You will see a matrix of metrics relating to the performance of your loops. II, which stands for initiation interval, is a measure corresponding to how often the pipeline built from your loop can be fed new data. Or, in other words, how often a new iteration of your loop can be launched. This metric is like golf, a low number is good, and 1 is the best you can do. Notice the number is 233. That is a very bad number, and something you will want to optimize if this is a loop that gets executed many times in your code. (In this particular piece of code, this loop is executed many times.)

<img src="Assets/loops_analysis.png">

__We won't go deeply into the optimization technique here (Shameless plug: take one of our free workshops to learn more about optimization when targetting FPGAs!), but moving the operation from operating on an accessor to a buffer to working on an array which gets implemented as on-chip memory in the FPGA will greatly improve this situation. Let's compile that improved code now and re-examine the report.__

This compile will take a couple of minutes.

In [ ]:
! dpcpp -fintelfpga lab/hough_transform_local_mem.cpp -fsycl-link=early -Xshardware -o bin/hough_transform_local_mem.a
! echo "The compile is finished."

__Run the following command to create a zip file that includes the report, like you did before.__

In [ ]:
! cd bin/hough_transform_local_mem.prj; zip -r ../../report_improved.zip reports > /dev/null
! echo "The zipped report file will appear in the file browser pane."

__Open up the report locally like you did before, and browse to the Loops Analysis section.__

The II metrics have gone down considerably. Our highest is now just 2 clock cycles. Much better! Can you do more to improve the kernel performance? (Yes! And shameless plug :) Attend a free workshop to learn more!)

The report is shown in the screenshot below.

<img src="Assets/improved_loops_report.png">

***
## Stage 3: Full Compile

**Hours of compilation**

<img src="Assets/full_compile.png">

The next step is to do a full compile of your code resulting in an executable that will contain an FPGA bitstream. When run, this executable will execute the portion of code within the kernel scope on the FPGA.

During this stage, you can
* Compile an FPGA bitstream for your design and run it on an FPGA
* Attain automated timing closure
* Obtain In-hardware verification
* Take advantage of Intel® VTune™ Profiler for real-time analysis of design.

The command to run this stage of the compilation is shown below.

<img src="Assets/full_compile_cmd.png">

Full compiles on the DevCloud must be done on nodes with the fpga_compile attribute. The command to do this is done below.

The build_fpga_hw_s10.sh script contains the command shown above within it. The qsub command is used to submit the job to the work queue of the DevCloud.

To learn more about running on the DevCloud including the types of nodes and more about the qsub command, see the Hello World! tutorial at <a href="https://devcloud.intel.com/oneapi/get_started/baseToolkitSamples/https://devcloud.intel.com/oneapi/get_started/baseToolkitSamples/"> this link</a>.

__This command will not be executed during the live demonstration due to time constraints. Please do this on your own when you have time.__
<br>
When you execute the cell below, it will give you a job ID as output.

In [ ]:
! qsub -l nodes=1:fpga_compile:ppn=2 -l walltime=24:00:00 -d . build_fpga_hw_s10.sh

__To check the status of the compilation kicked off above while it is running, execute the cell below.__<br>
When you execute the cell below, if you still see the job ID from the last code cell in the list, the job is still executing. If it is missing, the job is done.<br>
Check the files build_fpga_hw_s10.sh.o(job ID) and build_fpga_hw_s10.sh.e(job ID) to assess any errors that might occur.

In [ ]:
! qstat -n -1

## Now it's time to run the kernel on the FPGA!

To run on the FPGA, you must run on an fpga_runtime node with the correct board. Since the code was compiled using the default board, which is a board containing an Stratix 10 FPGA, we choose a Stratix10 board.

Since the code was compiled with the -Xsprofile switch, you can profile it using VTune. Read more about using VTune with FPGA in the <a href="https://www.intel.com/content/www/us/en/develop/documentation/oneapi-fpga-optimization-guide/top/analyze-your-design/analyze-the-fpga-image/intel-fpga-dynamic-profiler-for-dpc.html">Intel® oneAPI DPC++ FPGA Optimization Guide</a>.

__The command below will cause the compiled code to be run on a host with an FPGA attached, and the code in the kernel scope will run on the FPGA.__
<br>The command is submitted to the queue for the node using the qsub command. The shell script being run here simply contains a cd command to the directory, and a call to the executable. When you run the executable, the code in kernel scope is being run on the FPGA with the custom bitstream that was built from your code.<br>
When you execute the cell below, it will give you a job ID as output.

In [ ]:
! qsub -l nodes=1:fpga_runtime:stratix10:ppn=2 -d . run_fpga_hw_s10.sh

__To check the status of the compilation kicked off above while it is running, execute the cell below.__<br>
When you execute the cell below, if you still see the job ID from the last code cell in the list, the job is still executing. If it is missing, the job is done.<br>
Check the files build_fpga_hw_s10.sh.o(job ID) and build_fpga_hw_s10.sh.e(job ID) to assess any errors that might occur.

In [ ]:
! qstat -n -1

***
## To Learn More

The next best step in your learning is to work through the tutorials available at our <a href="https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/DPC%2B%2BFPGA">GitHub</a>. These are also available by running the **oneapi-cli** command within a terminal on a host where the Base Toolkit is installed.

Use the <a href="https://www.intel.com/content/www/us/en/developer/articles/code-sample/explore-dpcpp-through-intel-fpga-code-samples.html">Samples Navigation Guide</a> to know which tutorials to run as you progress in your learning, and the <a href="https://www.intel.com/content/www/us/en/develop/documentation/oneapi-fpga-optimization-guide/top.html"> Intel® oneAPI DPC++ FPGA Optimization Guide</a> to learn about the techniques more deeply.